In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
data = pd.read_csv('../input/wine-dataset-for-clustering/wine-clustering.csv')

In [ ]:
data.head()

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.isnull().sum().sort_values(ascending=False)

In [ ]:
sns.set(font_scale=1.3, rc={'figure.figsize':(20,20)})
ax=data.hist(bins=20,color='blue' )

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

data_scaled=data.copy()
data_scaled[data_scaled.columns]=scaler.fit_transform(data_scaled)

In [ ]:
data_scaled.describe()

In [ ]:
corr = data_scaled.corr()
plt.figure(figsize=(12,12))
sns.heatmap(corr, linewidths=.5, cmap='viridis')

## using PCA

In [ ]:
pca = PCA()

pca_data = pca.fit_transform(data_scaled)

In [ ]:
pca.get_covariance()

In [ ]:
explained_variance=pca.explained_variance_ratio_
explained_variance

In [ ]:
with plt.style.context('dark_background'):
    plt.figure(figsize=(8, 6))

    plt.bar(range(13), explained_variance, alpha=0.5, align='center',
            label='Explained Variance representation')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layout()

In [ ]:
#Plotting the Cumulative Summation of the Explained Variance
plt.figure(figsize = (8,5))
plt.plot(np.cumsum(pca.explained_variance_ratio_) * 100)
plt.xlabel('Number of Components')
plt.ylabel('Variance (%)')
plt.title('derma - Explained Variance')

# We want to preserve 95% of the variance so plot a horizontal line at 95% variance
plt.axhline(y=85, color='r', linestyle='--')
plt.show()

2 pc explain about 55% of the variance

## pca for 2 PC

In [ ]:
pca2 = PCA(2)

pca_data2 = pca2.fit_transform(data_scaled)

print ('Cumulative variance explained by 2 principal components: {:.2%}'.format(np.sum(pca2.explained_variance_ratio_)))

In [ ]:

plt.figure(figsize=(10,6))
plt.scatter(x=pca_data2[:, 0], y=pca_data2[:, 1], color='blue',lw=0.1)
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.title('Data')
plt.show()

### using  k-means

**Applying clustering on main dataset**

In [ ]:
SSE_scaled = []
from sklearn.cluster import KMeans

In [ ]:
for cluster in range(1,20):
    kmeans = KMeans( n_clusters = cluster)
    kmeans.fit(data_scaled)
    SSE_scaled.append(kmeans.inertia_)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(12,6))
frame_scaled = pd.DataFrame({'Cluster':range(1,20), 'SSE':SSE_scaled})
plt.plot(frame_scaled['Cluster'], frame_scaled['SSE'], marker='o')
plt.xlabel("Clusters")
plt.ylabel("sq. sum of error")

In [ ]:
kmeans = KMeans(n_clusters = 3,init='k-means++',random_state=9)
kmeans.fit(data_scaled)
pred = kmeans.predict(data_scaled)

In [ ]:
frame = pd.DataFrame(data_scaled)
data['cluster'] = pred

data['cluster'].value_counts()

In [ ]:
aux=data.columns.tolist()
aux[0:len(aux)-1]

for cluster in aux[0:len(aux)-1]:
    grid= sns.FacetGrid(data, col='cluster')
    grid.map(plt.hist, cluster,color='red')